In [1]:
import pandas as pd
import scipy.stats as stats
import numpy as np
from sqlalchemy import create_engine

In [2]:
url = "https://heartdisease4.s3.us-east-2.amazonaws.com/heart.csv"

df = pd.read_csv(url)
df.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1


In [3]:
#from sklearn.feature_selection import SelectKBest
#from sklearn.feature_selection import chi2

#X1 = df.iloc[:,0:-1]
#y1 = df.iloc[:,-1]
#bestfeatures = SelectKBest(score_func=chi2, k='all')
#fit = bestfeatures.fit(X1,y1)
#dfscores = pd.DataFrame(fit.scores_)
#dfcolumns = pd.DataFrame(X1.columns)

#featureScores = pd.concat([dfcolumns,dfscores],axis=1)
#featureScores.columns = ['Specs','Score']  #naming the dataframe columns
#print(featureScores.nlargest(10,'Score'))  #print 10 best features

In [4]:
df = df[['thalach',
        'oldpeak',
         'ca',
         'cp',
         'exang',
         'chol',
         'age',
         'trestbps',
         'slope',
         'sex',
         'target'
        ]]

In [5]:
new_df = pd.get_dummies(df, columns=['ca',
                                     'cp',
                                    'exang',
                                    'sex',
                                    'slope'])

#### Min Max Scaling

In [6]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
df['age'] = scaler.fit_transform(df['age'].values.reshape(-1,1))
df['trestbps'] = scaler.fit_transform(df['trestbps'].values.reshape(-1,1))
df['chol'] = scaler.fit_transform(df['chol'].values.reshape(-1,1))
new_df['oldpeak'] = scaler.fit_transform(new_df['oldpeak'].values.reshape(-1,1))
new_df['thalach'] = scaler.fit_transform(new_df['thalach'].values.reshape(-1,1))

### Models

In [7]:
from sklearn.model_selection import train_test_split

X = new_df.drop('target', axis=1)
y = new_df['target']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [8]:
from sklearn.neighbors import KNeighborsClassifier

train_scores = []
test_scores = []

for k in range(1, 33, 2):
    knn = KNeighborsClassifier(n_neighbors = k)
    knn.fit(X_train, y_train)
    train_score = knn.score(X_train, y_train)
    test_score = knn.score(X_test, y_test)
    train_scores.append(train_score)
    test_scores.append(test_score)
    print(f'k: {k}, Train/Test Score: {train_score:0.3f} / {test_score:.3f}')

k: 1, Train/Test Score: 1.000 / 0.513
k: 3, Train/Test Score: 0.758 / 0.579
k: 5, Train/Test Score: 0.722 / 0.526
k: 7, Train/Test Score: 0.696 / 0.526
k: 9, Train/Test Score: 0.639 / 0.579
k: 11, Train/Test Score: 0.648 / 0.605
k: 13, Train/Test Score: 0.621 / 0.645
k: 15, Train/Test Score: 0.617 / 0.645
k: 17, Train/Test Score: 0.586 / 0.632
k: 19, Train/Test Score: 0.595 / 0.632
k: 21, Train/Test Score: 0.599 / 0.632
k: 23, Train/Test Score: 0.608 / 0.618
k: 25, Train/Test Score: 0.612 / 0.658
k: 27, Train/Test Score: 0.595 / 0.658
k: 29, Train/Test Score: 0.586 / 0.658
k: 31, Train/Test Score: 0.590 / 0.658


In [9]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score, GridSearchCV

gsc = GridSearchCV(estimator=RandomForestClassifier(),
                  param_grid={
                      'max_depth': range(3,7),
                      'n_estimators': (10, 50, 100, 1000),},
                   cv=5, scoring='neg_mean_squared_error', verbose=0, n_jobs=-1)

grid_result = gsc.fit(X_train, y_train)
best_params = grid_result.best_params_

C:\Users\mcand\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:813: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


In [10]:
rfr = RandomForestClassifier(max_depth=best_params['max_depth'],
                            n_estimators=best_params['n_estimators'],
                            random_state=42, verbose=False)

rfr.fit(X_train, y_train)
rfr.score(X_train, y_train)
rfr.score(X_test, y_test)

0.881578947368421

In [14]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense

def build_classifier():
    regressor = Sequential()
    regressor.add(Dense(units=13, input_dim=21))
    regressor.add(Dense(units=1))
    regressor.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae', 'accuracy'])
    return regressor

In [15]:
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier

classifier = KerasClassifier(build_fn = build_classifier, batch_size=32, epochs=1000)

In [16]:
results = classifier.fit(X_train, y_train)

Epoch 1/1000
227/227 [==============================] - 0s 496us/sample - loss: 22444.8107 - mean_absolute_error: 148.7676 - acc: 0.4537
Epoch 2/1000
227/227 [==============================] - 0s 31us/sample - loss: 16202.4149 - mean_absolute_error: 126.3293 - acc: 0.4537
Epoch 3/1000
227/227 [==============================] - 0s 31us/sample - loss: 11109.4226 - mean_absolute_error: 104.4115 - acc: 0.4537
Epoch 4/1000
227/227 [==============================] - 0s 35us/sample - loss: 7175.3377 - mean_absolute_error: 83.5625 - acc: 0.4537
Epoch 5/1000
227/227 [==============================] - 0s 40us/sample - loss: 4340.6169 - mean_absolute_error: 64.5103 - acc: 0.4537
Epoch 6/1000
227/227 [==============================] - 0s 31us/sample - loss: 2467.3444 - mean_absolute_error: 47.8292 - acc: 0.4581
Epoch 7/1000
227/227 [==============================] - 0s 40us/sample - loss: 1324.4532 - mean_absolute_error: 33.7301 - acc: 0.4581
Epoch 8/1000
227/227 [==============================] -

227/227 [==============================] - 0s 31us/sample - loss: 102.3748 - mean_absolute_error: 7.8717 - acc: 0.4493
Epoch 63/1000
227/227 [==============================] - 0s 40us/sample - loss: 99.9055 - mean_absolute_error: 7.7957 - acc: 0.4405
Epoch 64/1000
227/227 [==============================] - 0s 31us/sample - loss: 97.0990 - mean_absolute_error: 7.7044 - acc: 0.4493
Epoch 65/1000
227/227 [==============================] - 0s 114us/sample - loss: 93.9578 - mean_absolute_error: 7.5624 - acc: 0.4537
Epoch 66/1000
227/227 [==============================] - 0s 31us/sample - loss: 91.6648 - mean_absolute_error: 7.4420 - acc: 0.4493
Epoch 67/1000
227/227 [==============================] - 0s 26us/sample - loss: 89.4541 - mean_absolute_error: 7.3455 - acc: 0.4493
Epoch 68/1000
227/227 [==============================] - 0s 31us/sample - loss: 86.5874 - mean_absolute_error: 7.2337 - acc: 0.4449
Epoch 69/1000
227/227 [==============================] - 0s 31us/sample - loss: 83.4983 

Epoch 124/1000
227/227 [==============================] - 0s 26us/sample - loss: 12.4286 - mean_absolute_error: 2.7726 - acc: 0.4537
Epoch 125/1000
227/227 [==============================] - 0s 31us/sample - loss: 11.9361 - mean_absolute_error: 2.7224 - acc: 0.4493
Epoch 126/1000
227/227 [==============================] - ETA: 0s - loss: 7.4769 - mean_absolute_error: 2.2127 - acc: 0.562 - 0s 48us/sample - loss: 11.5496 - mean_absolute_error: 2.6980 - acc: 0.4449
Epoch 127/1000
227/227 [==============================] - 0s 31us/sample - loss: 11.1813 - mean_absolute_error: 2.6550 - acc: 0.4537
Epoch 128/1000
227/227 [==============================] - 0s 31us/sample - loss: 10.7575 - mean_absolute_error: 2.5986 - acc: 0.4449
Epoch 129/1000
227/227 [==============================] - 0s 44us/sample - loss: 10.4155 - mean_absolute_error: 2.5497 - acc: 0.4493
Epoch 130/1000
227/227 [==============================] - 0s 35us/sample - loss: 10.0361 - mean_absolute_error: 2.5040 - acc: 0.4493
E

Epoch 185/1000
227/227 [==============================] - 0s 31us/sample - loss: 1.7814 - mean_absolute_error: 1.0731 - acc: 0.4890
Epoch 186/1000
227/227 [==============================] - 0s 31us/sample - loss: 1.7086 - mean_absolute_error: 1.0500 - acc: 0.4890
Epoch 187/1000
227/227 [==============================] - 0s 31us/sample - loss: 1.6632 - mean_absolute_error: 1.0351 - acc: 0.5066
Epoch 188/1000
227/227 [==============================] - 0s 31us/sample - loss: 1.6389 - mean_absolute_error: 1.0261 - acc: 0.5110
Epoch 189/1000
227/227 [==============================] - 0s 31us/sample - loss: 1.6005 - mean_absolute_error: 1.0129 - acc: 0.5110
Epoch 190/1000
227/227 [==============================] - 0s 35us/sample - loss: 1.5724 - mean_absolute_error: 1.0030 - acc: 0.5066
Epoch 191/1000
227/227 [==============================] - 0s 31us/sample - loss: 1.5141 - mean_absolute_error: 0.9891 - acc: 0.5022
Epoch 192/1000
227/227 [==============================] - 0s 31us/sample - l

227/227 [==============================] - 0s 26us/sample - loss: 0.6926 - mean_absolute_error: 0.6749 - acc: 0.5771
Epoch 247/1000
227/227 [==============================] - 0s 26us/sample - loss: 0.6819 - mean_absolute_error: 0.6682 - acc: 0.5991
Epoch 248/1000
227/227 [==============================] - 0s 123us/sample - loss: 0.6869 - mean_absolute_error: 0.6642 - acc: 0.5903
Epoch 249/1000
227/227 [==============================] - 0s 26us/sample - loss: 0.6686 - mean_absolute_error: 0.6602 - acc: 0.5815
Epoch 250/1000
227/227 [==============================] - 0s 44us/sample - loss: 0.6637 - mean_absolute_error: 0.6614 - acc: 0.5991
Epoch 251/1000
227/227 [==============================] - 0s 26us/sample - loss: 0.6527 - mean_absolute_error: 0.6541 - acc: 0.5947
Epoch 252/1000
227/227 [==============================] - 0s 31us/sample - loss: 0.6504 - mean_absolute_error: 0.6488 - acc: 0.5903
Epoch 253/1000
227/227 [==============================] - 0s 31us/sample - loss: 0.6477 - 

...

227/227 [==============================] - 0s 26us/sample - loss: 0.1396 - mean_absolute_error: 0.3012 - acc: 0.8282
Epoch 743/1000
227/227 [==============================] - 0s 31us/sample - loss: 0.1553 - mean_absolute_error: 0.3192 - acc: 0.7885
Epoch 744/1000
227/227 [==============================] - 0s 31us/sample - loss: 0.1324 - mean_absolute_error: 0.2887 - acc: 0.8194
Epoch 745/1000
227/227 [==============================] - 0s 26us/sample - loss: 0.1331 - mean_absolute_error: 0.2960 - acc: 0.8238
Epoch 746/1000
227/227 [==============================] - 0s 44us/sample - loss: 0.1484 - mean_absolute_error: 0.3030 - acc: 0.8062
Epoch 747/1000
227/227 [==============================] - 0s 35us/sample - loss: 0.1372 - mean_absolute_error: 0.2966 - acc: 0.8238
Epoch 748/1000
227/227 [==============================] - 0s 35us/sample - loss: 0.1334 - mean_absolute_error: 0.2958 - acc: 0.8194
Epoch 749/1000
227/227 [==============================] - 0s 35us/sample - loss: 0.1394 - m

227/227 [==============================] - 0s 31us/sample - loss: 0.1365 - mean_absolute_error: 0.3001 - acc: 0.8194
Epoch 805/1000
227/227 [==============================] - 0s 26us/sample - loss: 0.1272 - mean_absolute_error: 0.2894 - acc: 0.8282
Epoch 806/1000
227/227 [==============================] - 0s 35us/sample - loss: 0.1251 - mean_absolute_error: 0.2862 - acc: 0.8590
Epoch 807/1000
227/227 [==============================] - 0s 31us/sample - loss: 0.1350 - mean_absolute_error: 0.2947 - acc: 0.8370
Epoch 808/1000
227/227 [==============================] - 0s 31us/sample - loss: 0.1313 - mean_absolute_error: 0.2914 - acc: 0.8326
Epoch 809/1000
227/227 [==============================] - 0s 31us/sample - loss: 0.1535 - mean_absolute_error: 0.3114 - acc: 0.7885
Epoch 810/1000
227/227 [==============================] - 0s 31us/sample - loss: 0.1343 - mean_absolute_error: 0.2986 - acc: 0.8370
Epoch 811/1000
227/227 [==============================] - 0s 31us/sample - loss: 0.1265 - m

227/227 [==============================] - 0s 31us/sample - loss: 0.1375 - mean_absolute_error: 0.2967 - acc: 0.8414
Epoch 867/1000
227/227 [==============================] - 0s 26us/sample - loss: 0.1453 - mean_absolute_error: 0.3071 - acc: 0.8282
Epoch 868/1000
227/227 [==============================] - 0s 26us/sample - loss: 0.1230 - mean_absolute_error: 0.2840 - acc: 0.8238
Epoch 869/1000
227/227 [==============================] - 0s 26us/sample - loss: 0.1288 - mean_absolute_error: 0.2876 - acc: 0.8326
Epoch 870/1000
227/227 [==============================] - 0s 26us/sample - loss: 0.1442 - mean_absolute_error: 0.3060 - acc: 0.8194
Epoch 871/1000
227/227 [==============================] - 0s 40us/sample - loss: 0.1609 - mean_absolute_error: 0.3197 - acc: 0.8194
Epoch 872/1000
227/227 [==============================] - 0s 31us/sample - loss: 0.1834 - mean_absolute_error: 0.3441 - acc: 0.7974
Epoch 873/1000
227/227 [==============================] - 0s 35us/sample - loss: 0.1225 - m

227/227 [==============================] - 0s 26us/sample - loss: 0.1199 - mean_absolute_error: 0.2789 - acc: 0.8502
Epoch 929/1000
227/227 [==============================] - 0s 31us/sample - loss: 0.1201 - mean_absolute_error: 0.2768 - acc: 0.8502
Epoch 930/1000
227/227 [==============================] - 0s 31us/sample - loss: 0.1238 - mean_absolute_error: 0.2850 - acc: 0.8458
Epoch 931/1000
227/227 [==============================] - 0s 26us/sample - loss: 0.1283 - mean_absolute_error: 0.2904 - acc: 0.8326
Epoch 932/1000
227/227 [==============================] - 0s 31us/sample - loss: 0.1268 - mean_absolute_error: 0.2856 - acc: 0.8458
Epoch 933/1000
227/227 [==============================] - 0s 31us/sample - loss: 0.1754 - mean_absolute_error: 0.3439 - acc: 0.7841
Epoch 934/1000
227/227 [==============================] - 0s 31us/sample - loss: 0.1196 - mean_absolute_error: 0.2752 - acc: 0.8546
Epoch 935/1000
227/227 [==============================] - 0s 31us/sample - loss: 0.1228 - m

227/227 [==============================] - 0s 119us/sample - loss: 0.1419 - mean_absolute_error: 0.3001 - acc: 0.8062
Epoch 991/1000
227/227 [==============================] - 0s 26us/sample - loss: 0.1629 - mean_absolute_error: 0.3304 - acc: 0.8062
Epoch 992/1000
227/227 [==============================] - 0s 26us/sample - loss: 0.1472 - mean_absolute_error: 0.2966 - acc: 0.8150
Epoch 993/1000
227/227 [==============================] - 0s 26us/sample - loss: 0.1207 - mean_absolute_error: 0.2847 - acc: 0.8590
Epoch 994/1000
227/227 [==============================] - 0s 35us/sample - loss: 0.1223 - mean_absolute_error: 0.2814 - acc: 0.8414
Epoch 995/1000
227/227 [==============================] - 0s 31us/sample - loss: 0.1590 - mean_absolute_error: 0.3252 - acc: 0.7974
Epoch 996/1000
227/227 [==============================] - 0s 31us/sample - loss: 0.1693 - mean_absolute_error: 0.3340 - acc: 0.7797
Epoch 997/1000
227/227 [==============================] - 0s 31us/sample - loss: 0.1392 - 